In [34]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import os
import pickle
import sys

import numpy as np
import pandas as pd
import seaborn as sns
# testing
from pandas.testing import assert_frame_equal
from tqdm import tqdm

import j_utils.munging as mg
from lendingclub.lc_utils import gen_datasets
from lendingclub import config

pd.options.display.max_columns = 999
pd.options.display.max_rows = 60
pd.options.display.max_seq_items = None

# Testing scripts below

In [82]:
from lendingclub.modeling import models

In [83]:
df = pd.read_feather(os.path.join(config.data_dir, 'test_base_loan_info.fth'))
df.shape

(1181, 90)

In [84]:
mpath = config.modeling_dir

In [85]:
ls {mpath}

A_model_dataproc.pkl         C_model_dataproc.pkl  F_model_dataproc.pkl
A_model.pkl                  C_model.pkl           F_model.pkl
baseline_model_dataproc.pkl  D_model_dataproc.pkl  G_model_dataproc.pkl
baseline_model.pkl           D_model.pkl           G_model.pkl
B_model_dataproc.pkl         E_model_dataproc.pkl
B_model.pkl                  E_model.pkl


In [132]:
m = models.Model('B')
m.m

42

In [4]:
all_scores = pd.read_feather(os.path.join(config.data_dir, 'all_eval_loan_info_scored.fth'))
all_scores.head()

,end_d,issue_d,maturity_paid,maturity_time,maturity_time_stat_adj,maturity_paid_stat_adj,rem_to_be_paid,roi_simple,target_loose,target_strict,loan_status,id,grade,term,0.05,0.06,0.07,0.08,0.09,0.1,0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,0.19,0.2,0.21,0.22,0.23,0.24,0.25,0.26,0.27,0.28,0.29,0.3,0.31,0.32,0.33,0.34,0.35,catboost_regr_score,catboost_clf_score
0,2010-12-01,2007-12-01,1.000000,1.0,1.0,1.0,0.000000,1.123967,0,0,paid,166683,A,36,0.041716,0.026271,0.011143,-0.003676,-0.018194,-0.032417,-0.046353,-0.060008,-0.073389,-0.086502,-0.099354,-0.111950,-0.124297,-0.136400,-0.148265,-0.159897,-0.171302,-0.182486,-0.193452,-0.204207,-0.214754,-0.225099,-0.235246,-0.245199,-0.254964,-0.264544,-0.273943,-0.283166,-0.292216,-0.301097,-0.309814,-0.039020,0.424992
1,2010-02-01,2007-12-01,1.000000,1.0,1.0,1.0,0.000000,1.141801,0,0,paid,174377,B,36,0.065560,0.051043,0.036788,0.022790,0.009042,-0.004460,-0.017721,-0.030746,-0.043539,-0.056106,-0.068451,-0.080578,-0.092492,-0.104196,-0.115696,-0.126996,-0.138098,-0.149008,-0.159728,-0.170264,-0.180617,-0.190793,-0.200794,-0.210624,-0.220286,-0.229784,-0.239121,-0.248299,-0.257323,-0.266194,-0.274917,-0.058365,0.574708
2,2010-12-01,2007-12-01,1.000000,1.0,1.0,1.0,0.000000,1.237031,0,0,paid,178446,E,36,0.146518,0.129520,0.112871,0.096562,0.080585,0.064931,0.049594,0.034566,0.019840,0.005408,-0.008735,-0.022598,-0.036187,-0.049507,-0.062565,-0.075367,-0.087919,-0.100227,-0.112296,-0.124132,-0.135740,-0.147125,-0.158293,-0.169247,-0.179994,-0.190538,-0.200882,-0.211032,-0.220993,-0.230767,-0.240360,-0.076193,0.782984
3,2008-12-01,2008-01-01,0.191822,1.0,1.0,1.0,14147.749023,0.246457,1,1,charged_off,209667,E,36,-0.758478,-0.759404,-0.760324,-0.761238,-0.762146,-0.763049,-0.763946,-0.764837,-0.765723,-0.766603,-0.767477,-0.768346,-0.769210,-0.770068,-0.770921,-0.771768,-0.772610,-0.773447,-0.774279,-0.775106,-0.775928,-0.776744,-0.777556,-0.778362,-0.779164,-0.779961,-0.780753,-0.781540,-0.782322,-0.783100,-0.783873,-0.046314,0.673052
4,2011-02-01,2008-01-01,1.000000,1.0,1.0,1.0,0.000000,1.262633,0,0,paid,223015,F,36,0.170240,0.152890,0.135895,0.119247,0.102938,0.086960,0.071305,0.055965,0.040933,0.026202,0.011765,-0.002386,-0.016256,-0.029852,-0.043181,-0.056249,-0.069061,-0.081625,-0.093944,-0.106025,-0.117874,-0.129495,-0.140894,-0.152076,-0.163045,-0.173807,-0.184366,-0.194727,-0.204893,-0.214871,-0.224662,-0.074388,0.470468
